In [1]:
%%bash 
which python

/global/project/projectdirs/atlas/xju/miniconda3/envs/py3.6/bin/python


In [2]:
import numpy as np
import pandas as pd
import time
import sklearn.metrics

from trackml.dataset import load_event

import networkx as nx

In [3]:
import sys
sys.path.append('..')

from datasets.graph import load_graph

## Prepare network-x graphs using hitsgraph

In [4]:
base_dir = '/global/cscratch1/sd/xju/heptrkx/data/hitgraphs_001/event00000{}_g{:03d}.npz'

In [5]:
def get_edge_features(in_node, out_node):
    # input are the features of incoming and outgoing nodes
    # they are ordered as [r, phi, z]
    in_r, in_phi, _   = in_node
    out_r, out_phi, _ = out_node
    in_x = in_r * np.cos(in_phi)
    in_y = in_r * np.sin(in_phi)
    out_x = out_r * np.cos(out_phi)
    out_y = out_r * np.sin(out_phi)
    return np.sqrt((in_x - out_x)**2 + (in_y - out_y)**2)

In [6]:
def networkx_graph_from_hitsgraph(ievt, isec):
    ## ievt start from 1000
    file_name = base_dir.format(ievt, isec)
    return hitsgraph_to_networkx_graph(load_graph(file_name))


def hitsgraph_to_networkx_graph(G):
    n_nodes, n_edges = G.Ri.shape
    
    graph = nx.DiGraph()

    ## add nodes
    for i in range(n_nodes):
        graph.add_node(i, pos=G.X[i], solution=0.0)
    
    for iedge in range(n_edges):
        in_node_id  = G.Ri[:, iedge].nonzero()[0][0]
        out_node_id = G.Ro[:, iedge].nonzero()[0][0]

        # distance as features
        in_node_features  = G.X[in_node_id]
        out_node_features = G.X[out_node_id]
        distance = get_edge_features(in_node_features, out_node_features)
        # add edges, bi-directions
        graph.add_edge(in_node_id, out_node_id, distance=distance, solution=G.y[iedge])
        graph.add_edge(out_node_id, in_node_id, distance=distance, solution=G.y[iedge])
        # add "solution" to nodes
        graph.node[in_node_id].update(solution=G.y[iedge])
        graph.node[out_node_id].update(solution=G.y[iedge])
        
        
    # add global features, not used for now
    graph.graph['features'] = np.array([0.])
    
    return graph

def graph_to_input_target(graph):
    def create_feature(attr, fields):
        return np.hstack([np.array(attr[field], dtype=float) for field in fields])
    
    input_node_fields = ("pos",)
    input_edge_fields = ("distance",)
    target_node_fields = ("solution",)
    target_edge_fields = ("solution",)
    
    input_graph = graph.copy()
    target_graph = graph.copy()
    
    for node_index, node_feature in graph.nodes(data=True):
        input_graph.add_node(
            node_index, features=create_feature(node_feature, input_node_fields)
        )
        target_graph.add_node(
            node_index, features=create_feature(node_feature, target_node_fields)
        )
        
    for receiver, sender, features in graph.edges(data=True):
        input_graph.add_edge(
            sender, receiver, features=create_feature(features, input_edge_fields)
        )
        target_graph.add_edge(
            sender, receiver, features=create_feature(features, target_edge_fields)
        )
        
    input_graph.graph['features'] = input_graph.graph['features'] = np.array([0.0])
    return input_graph, target_graph


def generate_input_target(n_graphs, start_evt_id=1000):
    input_graphs = []
    target_graphs = []
    for i in range(n_graphs):
        evt_id = start_evt_id + i
        isec = 0
        graph = networkx_graph_from_hitsgraph(evt_id, isec)
        input_graph, output_graph = graph_to_input_target(graph)
        input_graphs.append(input_graph)
        target_graphs.append(output_graph)
    return input_graphs, target_graphs


## Use interaction network from modules

In [7]:
import tensorflow as tf

In [8]:
from graph_nets import modules
from graph_nets import utils_tf
from graph_nets import utils_np

import sonnet as snt

In [9]:
tf.reset_default_graph()

### Define GNN model

In [10]:
NUM_LAYERS = 2  # Hard-code number of layers in the edge/node/global models.
LATENT_SIZE = 16  # Hard-code latent layer sizes for demos.


def make_mlp_model():
  """Instantiates a new MLP, followed by LayerNorm.

  The parameters of each new MLP are not shared with others generated by
  this function.

  Returns:
    A Sonnet module which contains the MLP and LayerNorm.
  """
  return snt.Sequential([
      snt.nets.MLP([LATENT_SIZE] * NUM_LAYERS, activate_final=True),
      snt.LayerNorm()
  ])

class MLPGraphIndependent(snt.AbstractModule):
  """GraphIndependent with MLP edge, node, and global models."""

  def __init__(self, name="MLPGraphIndependent"):
    super(MLPGraphIndependent, self).__init__(name=name)
    with self._enter_variable_scope():
      self._network = modules.GraphIndependent(
          edge_model_fn=make_mlp_model,
          node_model_fn=make_mlp_model,
          global_model_fn=None)

  def _build(self, inputs):
    return self._network(inputs)



class SegmentClassifier(snt.AbstractModule):

  def __init__(self, name="SegmentClassifier"):
    super(SegmentClassifier, self).__init__(name=name)

    self._encoder = MLPGraphIndependent()
    self._core = modules.InteractionNetwork(
        edge_model_fn=make_mlp_model,
        node_model_fn=make_mlp_model,
        reducer=tf.unsorted_segment_prod
    )

    # Transforms the outputs into the appropriate shapes.
    edge_output_size = 1
#     edge_fn = lambda: snt.Linear(edge_output_size, name="edge_output")
    edge_fn =lambda: snt.nets.MLP([edge_output_size], activation=tf.nn.tanh, name='edge_output')

    with self._enter_variable_scope():
      self._output_transform = modules.GraphIndependent(edge_fn, None, None)

  def _build(self, input_op, num_processing_steps):
    latent = self._encoder(input_op)

    output_ops = []
    for _ in range(num_processing_steps):
        core_input = latent
        latent = self._core(core_input)
        output_ops.append(self._output_transform(latent))
    return output_ops

In [11]:
model = SegmentClassifier()

### Write Loss functions and Feed-dict

In [12]:
global event_track
event_track = 1000
def create_feed_dict(batch_size, input_ph, target_ph):
    global event_track
    inputs, targets = generate_input_target(batch_size, event_track)
    event_track += batch_size

    input_graphs = utils_np.networkxs_to_graphs_tuple(inputs)
    target_graphs = utils_np.networkxs_to_graphs_tuple(targets)
    feed_dict = {input_ph: input_graphs, target_ph: target_graphs}
#         print(event_track)
    return feed_dict

In [13]:
def create_loss_ops(target_op, output_ops):
    # only use edges
    loss_ops = [
        tf.losses.sigmoid_cross_entropy(target_op.edges, output_op.edges)
        for output_op in output_ops
    ]
    return loss_ops


def make_all_runnable_in_session(*args):
  """Lets an iterable of TF graphs be output from a session as NP graphs."""
  return [utils_tf.make_runnable_in_session(a) for a in args]

In [14]:
def computer_matrics(target, output):
    tdds = utils_np.graphs_tuple_to_data_dicts(target)
    odds = utils_np.graphs_tuple_to_data_dicts(output)
    
    test_target = []
    test_pred = []
    for td, od in zip(tdds, odds):
        test_target.append(td['edges'])
        test_pred.append(od['edges'])
    
    test_target = np.concatenate(test_target, axis=0)
    test_pred   = np.concatenate(test_pred,   axis=0)
    
    thresh = 0.5
    y_pred, y_true = (test_pred > thresh), (test_target > thresh)
    return sklearn.metrics.precision_score(y_true, y_pred), sklearn.metrics.recall_score(y_true, y_pred)

In [15]:
batch_size = n_graphs = 2
num_training_iterations = 10000
num_processing_steps_tr = 4  ## level of message-passing

In [16]:
input_graphs, target_graphs = generate_input_target(n_graphs)
input_ph  = utils_tf.placeholders_from_networkxs(input_graphs, force_dynamic_num_graphs=True)
target_ph = utils_tf.placeholders_from_networkxs(target_graphs, force_dynamic_num_graphs=True)

In [17]:
output_ops_tr = model(input_ph, num_processing_steps_tr)

# Training loss.
loss_ops_tr = create_loss_ops(target_ph, output_ops_tr)
# Loss across processing steps.
loss_op_tr = sum(loss_ops_tr) / num_processing_steps_tr

# Optimizer
learning_rate = 1e-3
optimizer = tf.train.AdamOptimizer(learning_rate)
step_op = optimizer.minimize(loss_op_tr)

# Lets an iterable of TF graphs be output from a session as NP graphs.
input_ph, target_ph = make_all_runnable_in_session(input_ph, target_ph)

/global/project/projectdirs/atlas/xju/miniconda3/envs/py3.6/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [18]:
#@title Reset session  { form-width: "30%" }

# This cell resets the Tensorflow session, but keeps the same computational
# graph.

try:
  sess.close()
except NameError:
  pass
sess = tf.Session()
sess.run(tf.global_variables_initializer())

last_iteration = 0
logged_iterations = []
losses_tr = []
corrects_tr = []
solveds_tr = []


#@title Run training  { form-width: "30%" }

# You can interrupt this cell's training loop at any time, and visualize the
# intermediate results by running the next cell (below). You can then resume
# training by simply executing this cell again.

# How much time between logging and printing the current results.
log_every_seconds = 20

print("# (iteration number), T (elapsed seconds), "
      "Ltr (training loss), "
      "Precision, "
      "Recall")

start_time = time.time()
last_log_time = start_time
for iteration in range(last_iteration, num_training_iterations):
  last_iteration = iteration
  feed_dict = create_feed_dict(batch_size, input_ph, target_ph)
  train_values = sess.run({
      "step": step_op,
      "target": target_ph,
      "loss": loss_op_tr,
      "outputs": output_ops_tr
  }, feed_dict=feed_dict)
  the_time = time.time()
  elapsed_since_last_log = the_time - last_log_time

  if elapsed_since_last_log > log_every_seconds:
    last_log_time = the_time
    feed_dict = create_feed_dict(batch_size, input_ph, target_ph)
    test_values = sess.run({
        "target": target_ph,
        "loss": loss_op_tr,
        "outputs": output_ops_tr
    }, feed_dict=feed_dict)
    correct_tr, solved_tr = computer_matrics(
        test_values["target"], test_values["outputs"][-1])
    elapsed = time.time() - start_time
    losses_tr.append(train_values["loss"])
    corrects_tr.append(correct_tr)
    solveds_tr.append(solved_tr)
    logged_iterations.append(iteration)
    print("# {:05d}, T {:.1f}, Ltr {:.4f}, Lge {:.4f}, Precision {:.4f}, Recall"
          " {:.4f}".format(
              iteration, elapsed, train_values["loss"], test_values["loss"],
              correct_tr, solved_tr))

# (iteration number), T (elapsed seconds), Ltr (training loss), Precision, Recall
# 00005, T 26.0, Ltr 0.6362, Lge 0.6095, Precision 0.5758, Recall 0.0489
# 00010, T 51.6, Ltr 0.6143, Lge 0.6387, Precision 0.5152, Recall 0.0526
# 00018, T 76.2, Ltr 0.6131, Lge 0.5928, Precision 0.5609, Recall 0.0246
# 00023, T 96.0, Ltr 0.6434, Lge 0.6119, Precision 0.6460, Recall 0.0243
# 00029, T 117.0, Ltr 0.6223, Lge 0.5882, Precision 0.7229, Recall 0.0167
# 00035, T 138.5, Ltr 0.6184, Lge 0.6019, Precision 0.7286, Recall 0.0145
# 00041, T 160.7, Ltr 0.6109, Lge 0.5993, Precision 0.7368, Recall 0.0138
# 00047, T 183.6, Ltr 0.6025, Lge 0.6319, Precision 0.7677, Recall 0.0209
# 00053, T 205.9, Ltr 0.5919, Lge 0.6179, Precision 0.7569, Recall 0.0160
# 00059, T 226.5, Ltr 0.6028, Lge 0.6084, Precision 0.8364, Recall 0.0171
# 00066, T 249.5, Ltr 0.5623, Lge 0.6173, Precision 0.8354, Recall 0.0173
# 00072, T 270.2, Ltr 0.5933, Lge 0.5995, Precision 0.8424, Recall 0.0158
# 00078, T 290.7, Ltr 0.6027, Lge 

KeyboardInterrupt: 